In [ ]:
# STEP 1: Install required packages
!pip install -q streamlit pyngrok pandas

# STEP 2: Set your ngrok authtoken
from pyngrok import ngrok
ngrok.set_auth_token("2wbDASVryYnzh5OPDUJVzSfLLKM_6xcoryMdVGs9uoRDvsss5")

# STEP 3: Write a simple Streamlit app that loads your CSV
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import pandas as pd

st.set_page_config(layout="wide")
st.title("📊 Skill Epidemiology Dashboard")

# Load your metrics
df = pd.read_csv("/content/sample_data/epidemiological_skill_metrics.csv")

# Filters
revived = st.selectbox("Filter by Revived?", options=["All", "Yes", "No"])
mortality = st.selectbox("Filter by Mortality Risk", options=["All", "🟢", "🔵"])

if revived != "All":
    df = df[df["Revived?"] == revived]
if mortality != "All":
    df = df[df["Mortality Risk"] == mortality]

# Display table
st.dataframe(df)

# Plot: Top skills by Incidence
st.subheader("Top 10 Skills by Incidence (2023)")
top = df.sort_values(by="Incidence (2023)", ascending=False).head(10)
st.bar_chart(top.set_index("Skill")["Incidence (2023)"])
""")

# STEP 4: Start Streamlit in background
!streamlit run app.py &>/content/log.txt &

# STEP 5: Open the public URL via ngrok
public_url = ngrok.connect(8501)
print(f"✅ Streamlit app is live: {public_url}")


In [ ]:
# # STEP 1: Install required packages
# !pip install -q streamlit pyngrok pandas
from pyngrok import ngrok
import pandas as pd

# Set your ngrok token
ngrok.set_auth_token("2wbDASVryYnzh5OPDUJVzSfLLKM_6xcoryMdVGs9uoRDvsss5")

# Create Streamlit app
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import pandas as pd
import altair as alt

st.set_page_config(layout="wide")
st.title("📊 Skill Epidemiology Dashboard")

# Load metrics CSV
df = pd.read_csv("/content/sample_data/epidemiological_skill_metrics.csv")

# === Filters ===
st.sidebar.header("Filter")
revived_filter = st.sidebar.selectbox("Show Revived?", ["All", "Yes", "No"])
mortality_filter = st.sidebar.selectbox("Mortality Risk", ["All", "🟢", "🔵"])

if revived_filter != "All":
    df = df[df["Revived?"] == revived_filter]
if mortality_filter != "All":
    df = df[df["Mortality Risk"] == mortality_filter]

# === Main Table ===
st.subheader("🧠 Filtered Skill Epidemiology Table")
st.dataframe(df)

# === Charts ===
st.subheader("📈 Top 10 Skills by Incidence (2023)")
st.bar_chart(df.sort_values(by="Incidence (2023)", ascending=False).head(10).set_index("Skill")["Incidence (2023)"])

st.subheader("📉 Skills with Largest Decline")
st.bar_chart(df.sort_values(by="% Change in Incidence").head(10).set_index("Skill")["% Change in Incidence"])

st.subheader("♻️ Revived Skills")
revived_df = df[df["Revived?"] == "Yes"]
if not revived_df.empty:
    st.bar_chart(revived_df.sort_values(by="Incidence (2023)", ascending=False).head(10).set_index("Skill")["Incidence (2023)"])
else:
    st.write("No revived skills to show.")

# === Insights ===
st.subheader("📉 Dying but Historically Popular Skills")
dying_popular = df[(df["Mortality Risk"] == "🟢") & (df["Total Posts"] > 500)]
st.dataframe(dying_popular.sort_values(by="Total Posts", ascending=False).head(10))

st.subheader("🟦 Growing & Alive Skills")
survivors = df[(df["Mortality Risk"] == "🔵") & (df["% Change in Incidence"] > 0)]
st.dataframe(survivors.sort_values(by="% Change in Incidence", ascending=False).head(10))

# === Bubble Chart ===
st.subheader("🧪 Incidence vs Popularity (Bubble Chart)")
bubble = alt.Chart(df).mark_circle(size=80).encode(
    x='Incidence (2023):Q',
    y='Total Posts:Q',
    color='Mortality Risk:N',
    tooltip=['Skill', 'Incidence (2023)', 'Total Posts', 'Mortality Risk']
).interactive().properties(height=400)
st.altair_chart(bubble, use_container_width=True)

# === Download Button ===
csv = df.to_csv(index=False).encode('utf-8')
st.download_button("💾 Download Filtered Metrics", csv, "filtered_skills.csv", "text/csv")
""")

# Run Streamlit in background
!streamlit run app.py &>/content/log.txt &

# Open public tunnel
public_url = ngrok.connect(8501)
print(f"✅ Your Streamlit dashboard is live: {public_url}")


In [ ]:
from pyngrok import ngrok
import pandas as pd
import json
from datetime import datetime
from collections import defaultdict

# Set your ngrok token
ngrok.set_auth_token("2wbDASVryYnzh5OPDUJVzSfLLKM_6xcoryMdVGs9uoRDvsss5")

# === Write Streamlit App ===
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import altair as alt
from datetime import datetime
from collections import defaultdict
import json

st.set_page_config(layout="wide")
st.title("🧬 Skill Biology Dashboard")

# === Load and process JSON ===
with open("/content/sample_data/Posts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

posts = data['posts']['row']
skill_data = defaultdict(list)

for post in posts:
    tags_str = post.get('@Tags', '')
    tags = tags_str.strip('|').split('|') if tags_str else []
    creation_date_str = post.get('@CreationDate')
    view_count = int(post.get('@ViewCount', 0))
    score = int(post.get('@Score', 0))
    answer_count = int(post.get('@AnswerCount', 0))

    if creation_date_str:
        creation_date = datetime.strptime(creation_date_str, '%Y-%m-%dT%H:%M:%S.%f')
    else:
        continue

    for tag in tags:
        skill_data[tag].append({
            'creation_date': creation_date,
            'view_count': view_count,
            'score': score,
            'answer_count': answer_count
        })

metrics = []
for skill, entries in skill_data.items():
    dates = [entry['creation_date'] for entry in entries]
    views = [entry['view_count'] for entry in entries]
    scores = [entry['score'] for entry in entries]
    answers = [entry['answer_count'] for entry in entries]

    date_of_birth = min(dates).date()
    df = pd.DataFrame(entries)
    df['month'] = df['creation_date'].dt.to_period('M')
    peak_month = df.groupby('month').size().idxmax().strftime('%Y-%m')

    avg_views = sum(views) / len(views) if views else 0
    avg_score = sum(scores) / len(scores) if scores else 0
    avg_answers = sum(answers) / len(answers) if answers else 0
    total_posts = len(entries)

    if avg_score >= 10 and avg_answers >= 3:
        immunity = 'High'
    elif avg_score >= 5:
        immunity = 'Medium'
    else:
        immunity = 'Low'

    metrics.append({
        'Skill': skill,
        'Date of Birth': date_of_birth,
        'Peak Activity Date': peak_month,
        'Avg Views': round(avg_views, 2),
        'Avg Score': round(avg_score, 2),
        'Avg Answers': round(avg_answers, 2),
        'Total Posts': total_posts,
        'Immunity Score': immunity
    })

df_metrics = pd.DataFrame(metrics)

# === Dashboard ===
st.subheader("📋 Skill Biology Table")
st.dataframe(df_metrics)

st.subheader("🔥 Top Skills by Total Posts")
top_skills = df_metrics.sort_values(by='Total Posts', ascending=False).head(10)
st.bar_chart(top_skills.set_index("Skill")["Total Posts"])

st.subheader("🧪 Views vs. Score (Bubble Chart)")
chart = alt.Chart(df_metrics).mark_circle(size=80).encode(
    x='Avg Views',
    y='Avg Score',
    color='Immunity Score',
    tooltip=['Skill', 'Avg Views', 'Avg Score', 'Avg Answers', 'Total Posts']
).interactive()
st.altair_chart(chart, use_container_width=True)
""")


# Launch public URL
public_url = ngrok.connect(8501)
print(f"✅ Your Skill Biology Dashboard is live: {public_url}")



In [ ]:
# Streamlit app to show inverse trend analysis between skills (no survival model)

!pip install -q streamlit pyngrok pandas altair scikit-learn

from pyngrok import ngrok
import pandas as pd
import numpy as np
import json
import re
from datetime import datetime
from collections import defaultdict
from itertools import combinations
from sklearn.linear_model import LinearRegression
import altair as alt

# Set ngrok token
ngrok.set_auth_token("2wbDASVryYnzh5OPDUJVzSfLLKM_6xcoryMdVGs9uoRDvsss5")

# === Write app.py ===
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
import re
import json
from datetime import datetime
from collections import defaultdict
from itertools import combinations
from sklearn.linear_model import LinearRegression
import altair as alt

st.set_page_config(layout="wide")
st.title("🔁 Skill Trend Inversion Dashboard")

# === Load JSON ===
with open("/content/sample_data/Posts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

posts = data['posts']['row']
tag_dates = defaultdict(list)

# === Extract tag dates ===
for post in posts:
    try:
        date = datetime.strptime(post["@CreationDate"], "%Y-%m-%dT%H:%M:%S.%f")
        tags = re.findall(r'\\|([^|]+)\\|', post.get("@Tags", ""))
        for tag in tags:
            tag_dates[tag].append(date)
    except:
        continue

# === Top 100 tags ===
tag_counts = {tag: len(dates) for tag, dates in tag_dates.items()}
top_100_tags = sorted(tag_counts, key=tag_counts.get, reverse=True)[:100]

# === Monthly series ===
tag_series = {}
for tag in top_100_tags:
    s = pd.Series(1, index=pd.to_datetime(tag_dates[tag]))
    s = s.resample("M").sum().fillna(0)
    tag_series[tag] = s

combined_index = pd.date_range(start="2008-01-01", end="2024-12-31", freq="M")
for tag in tag_series:
    tag_series[tag] = tag_series[tag].reindex(combined_index, fill_value=0)

# === UI: Pick skills to compare ===
st.sidebar.header("🧠 Skill Selection")
tag1 = st.sidebar.selectbox("Select Declining Skill", top_100_tags)
tag2 = st.sidebar.selectbox("Select Competing Skill", [t for t in top_100_tags if t != tag1])

# === Regression Function ===
def get_slope(ts):
    X = np.arange(len(ts)).reshape(-1, 1)
    y = ts.values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.coef_[0][0]

# === Process Selected Pair ===
s1 = tag_series[tag1]
s2 = tag_series[tag2]
mask = (s1 > 0) & (s2 > 0)
overlap = mask.sum()

if overlap < 6:
    st.warning("❌ Not enough overlap between skills for comparison (min 6 months).")
else:
    slope1 = get_slope(s1[mask])
    slope2 = get_slope(s2[mask])

    st.markdown(f"### {tag1} vs {tag2} — {overlap} months overlap")
    st.markdown(f"**{tag1} slope**: {slope1:.3f}, **{tag2} slope**: {slope2:.3f}")

    chart_df = pd.DataFrame({
        "Date": combined_index,
        tag1: s1.values,
        tag2: s2.values
    })
    chart_df = chart_df.melt("Date", var_name="Skill", value_name="Posts")

    chart = alt.Chart(chart_df).mark_line().encode(
        x='Date:T', y='Posts:Q', color='Skill:N'
    ).properties(height=400)

    st.altair_chart(chart, use_container_width=True)
""")

# === Launch app ===
public_url = ngrok.connect(8501)
print(f"✅ Streamlit Trend Inversion App is live: {public_url}")
